In [83]:
%reload_ext autoreload
%autoreload 2
import tensorflow as tf
import sys
import math
import numpy as np
sys.path.append('../../pytools')
import d2l

d2l.gpu_mem_init()

Memory growth enabled for PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'): True


In [84]:
def corr2d(x, k):
    height,width = k.shape[0],k.shape[1]
    y = tf.Variable(tf.zeros((x.shape[0] - height + 1, x.shape[1] - width + 1)))
    for i in range(y.shape[0]):
        for j in range(y.shape[1]):
            y[i,j].assign(tf.reduce_sum(x[i:i + height, j:j+width] * k))
    return y

In [85]:
X = tf.constant([[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]])
K = tf.constant([[0.0, 1.0], [2.0, 3.0]])
#corr2d(X, K)

In [86]:
class Conv2D(tf.keras.layers.Layer):
    def __init__(self):
        super().__init__()
    def build(self, shape):
        self.w = self.add_weight(name='weight', shape=shape, initializer=tf.random_normal_initializer())
        self.b = self.add_bias(name='bias', shape=(1,), initializer=tf.random_normal_initializer())
    def call(self, input):
        print(f'self.w.shape={self.w.shape}')
        return corr2d(inputs, self.w) + self.b

In [87]:
X = tf.Variable(tf.ones((6, 8)))
X[:, 2:6].assign(tf.zeros(X[:, 2:6].shape))

<tf.Variable 'UnreadVariable' shape=(6, 8) dtype=float32, numpy=
array([[1., 1., 0., 0., 0., 0., 1., 1.],
       [1., 1., 0., 0., 0., 0., 1., 1.],
       [1., 1., 0., 0., 0., 0., 1., 1.],
       [1., 1., 0., 0., 0., 0., 1., 1.],
       [1., 1., 0., 0., 0., 0., 1., 1.],
       [1., 1., 0., 0., 0., 0., 1., 1.]], dtype=float32)>

In [88]:
K = tf.constant([[1.0, -1.0]])
Y = corr2d(X, K)

conv2d = tf.keras.layers.Conv2D(1, (1,2), use_bias=False)

x = tf.reshape(X, (1,6,8,1))
y = tf.reshape(Y, (1,6,7,1))
lr = 3e-2
y_hat = conv2d(x)
#print(conv2d.get_weights())
sgd = tf.keras.optimizers.SGD(0.03)
for i in range(10):
    with tf.GradientTape() as t:
        y_hat = conv2d(x)
        l = (abs(y - y_hat))**2
        grads = t.gradient(l, conv2d.weights)
        sgd.apply_gradients(zip(grads, conv2d.trainable_variables))
        if (i + 1) % 2 == 0:
            print(f'epoch {i+1}, loss {tf.reduce_sum(l):.3f}')


epoch 2, loss 20.240
epoch 4, loss 3.723
epoch 6, loss 0.758
epoch 8, loss 0.182
epoch 10, loss 0.053
